In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
from nltk.tokenize import word_tokenize,wordpunct_tokenize
from collections import Counter
import json
import os
import torchtext
from collections import Counter,defaultdict
from nltk.tokenize import RegexpTokenizer
import re
import os
import json
from torchtext.vocab import GloVe
import random
import gc
from transformers import BertModel, BertTokenizer
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# Example text
# text = ["Hello, how are you doing today?","(n0)|(n1)#",'dj']

# Tokenize the text
# tokens = tokenizer.tokenize("(n0)|(n1)#500.2")
# print("Tokens:", tokens)

# Convert the tokens to token IDs
# token_ids = tokenizer.convert_tokens_to_ids(tokens)
# print("Token IDs:", token_ids)

# Add special tokens and create input IDs and attention mask
# input_ids = tokenizer.encode(text, add_special_tokens=True)
# print("Input IDs:", input_ids)
# tokenized_batch = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
# # Attention mask
# print(tokenized_batch,'bye')
# token = tokenizer.convert_ids_to_tokens([102])
# print(token,'hi')
# attention_mask = [1] * len(input_ids)
# print("Attention Mask:", attention_mask)
# idx_to_word = tokenizer.convert_ids_to_tokens
# print(idx_to_word(121))

In [ ]:
def preprocess_bert(data):
#     print(type(data[0]))
    problem = data['Problem']
    formula = data['linear_formula']
    ans = data['answer']
    max_len_Y = 0
    tokenized_problem = bert_tokenizer(problem, padding=True, truncation=True, return_tensors="pt")
    problem = tokenized_problem['input_ids']
    att_mask = tokenized_problem['attention_mask']
    for i in range(len(formula)):
        sentence = formula[i]
        sentence = sentence.lower()
        sentence = re.sub(re.escape('|'),' | ',sentence)
        tokens = word_tokenize(sentence)
        max_len_Y= max(max_len_Y,len(tokens)+2)
    for i in range(len(formula)):
        sentence = formula[i]
        sentence = sentence.lower()
        sentence = re.sub(re.escape('|'),' | ',sentence)
        tokens = word_tokenize(sentence)
        tokens = ['<start>'] + tokens + ['<end>']
        padding = ['<pad>']*(max_len_Y-len(tokens))
        tokens += padding
        token_idx = []
        for token in tokens:
            if(token in word_to_idx_Y):
                token_idx.append(word_to_idx_Y[token])
            else:
                token_idx.append(word_to_idx_Y['<unk>'])
        formula[i] = token_idx
    ans = torch.tensor(ans)
    formula = torch.tensor(formula)
    return problem,formula,ans,att_mask

In [ ]:
def load_data(file_path):
    with open(file_path+"/train.json",'r') as file:
        train_data=json.load(file)
    with open(file_path+"/test.json",'r') as file:
        test_data=json.load(file)
    with open(file_path+"/dev.json",'r') as file:
        val_data=json.load(file)
    return train_data,val_data,test_data
def build_vocab(data,typ):
    text = [example[typ] for example in data]
    vocab = Counter()
    vocab['<pad>'] = 0
    vocab['<start>'] = 0
    vocab['<end>'] = 0
    vocab['<unk>'] = 0
    max_len = 0
    for sentence in text:
        sentence = sentence.lower()
        sentence = re.sub(re.escape('|'),' | ',sentence)
        tokens = word_tokenize(sentence)
        vocab.update(tokens)
        max_len = max(len(tokens),max_len)
    idx_to_word = dict(enumerate(vocab))
    word_to_idx = {word:idx for idx,word in enumerate(vocab)}
    return vocab,idx_to_word,word_to_idx,max_len
def preprocess(data):
#     print(type(data[0]))
    problem = data['Problem']
    formula = data['linear_formula']
    ans = data['answer']
    max_len_X = 0
    max_len_Y = 0
    for i in range(len(problem)):
        sentence = problem[i]
        sentence = sentence.lower()
        sentence = re.sub(re.escape('|'),' | ',sentence)
        tokens = word_tokenize(sentence)
        max_len_X = max(max_len_X,len(tokens)+2)
    for i in range(len(formula)):
        sentence = formula[i]
        sentence = sentence.lower()
        sentence = re.sub(re.escape('|'),' | ',sentence)
        tokens = word_tokenize(sentence)
        max_len_Y= max(max_len_Y,len(tokens)+2)
    for i in range(len(formula)):
        sentence = problem[i]
        sentence = sentence.lower()
        sentence = re.sub(re.escape('|'),' | ',sentence)
        tokens = word_tokenize(sentence)
        tokens = ['<start>'] + tokens + ['<end>']
        padding = ['<pad>']*(max_len_X-len(tokens))
        tokens += padding
        token_idx = []
        for token in tokens:
            if(token in word_to_idx_X):
                token_idx.append(word_to_idx_X[token])
            else:
                token_idx.append(word_to_idx_Y['<unk>'])
        problem[i] = token_idx
    
        sentence = formula[i]
        sentence = sentence.lower()
        sentence = re.sub(re.escape('|'),' | ',sentence)
        tokens = word_tokenize(sentence)
        tokens = ['<start>'] + tokens + ['<end>']
        padding = ['<pad>']*(max_len_Y-len(tokens))
        tokens += padding
        token_idx = []
        for token in tokens:
            if(token in word_to_idx_Y):
                token_idx.append(word_to_idx_Y[token])
            else:
                token_idx.append(word_to_idx_Y['<unk>'])
        formula[i] = token_idx
    ans = torch.tensor(ans)
    problem = torch.tensor(problem)
    formula = torch.tensor(formula)
    return problem,formula,ans
# text = preprocess(test_data)
# print(text)

In [ ]:
train_data,val_data,test_data = load_data('/kaggle/input/text2math/data')

def data_loader(batch_size):    
    train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_data,batch_size = batch_size,shuffle = False)
    test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
    return train_dataloader,val_dataloader,test_dataloader

In [ ]:
vocab_X,idx_to_word_X,word_to_idx_X,max_len_X = build_vocab(train_data,'Problem')
vocab_Y,idx_to_word_Y,word_to_idx_Y,max_len_Y = build_vocab(train_data,'linear_formula')
# print(vocab_Y)
# print(idx_to_word_Y)
# print(word_to_idx_Y)
# print(len(vocab_X))

In [ ]:
class GloveEmbeddings():
    def __init__(self,embedding_dim,word_to_idx):
        self.embedding_dim = embedding_dim
        self.word_to_idx = word_to_idx
        self.vocab_size = len(word_to_idx)
    def get_embedding(self):
        glove = GloVe(name = '6B',dim = self.embedding_dim)
        embedding_matrix = torch.zeros((self.vocab_size,self.embedding_dim))
        embedding_matrix[0] = torch.zeros(self.embedding_dim) 
        embedding_matrix[1] = torch.randn(self.embedding_dim)
        embedding_matrix[2] = torch.randn(self.embedding_dim)
        embedding_matrix[3] = torch.randn(self.embedding_dim)
        
        for key,value in self.word_to_idx.items():
            if value in [0,1,2,3]:
                continue  
            else:
                if key in glove.stoi:
                    embedding_matrix[value] = torch.tensor(glove.vectors[glove.stoi[key]])
                else:
                    embedding_matrix[value] = torch.tensor(embedding_matrix[3])#for unk
        return embedding_matrix

In [ ]:
class LSTM_ENCODER(nn.Module):
    def __init__(self,input_size,embedding_dim,hidden_dim,num_layers,embedding_matrix):
        super(LSTM_ENCODER,self).__init__()
        self.input_size = input_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout = nn.Dropout(0.2)
        self.embedding_matrix = embedding_matrix
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix,padding_idx = 0)
        self.LSTM = nn.LSTM(self.embedding_dim,self.hidden_dim,num_layers = self.num_layers,bidirectional = True,dropout = 0.2,batch_first = True)
        self.hidden = nn.Linear(2*self.hidden_dim,self.hidden_dim)
        self.cell = nn.Linear(2*self.hidden_dim,self.hidden_dim)
    def forward(self,x,att_mask = None):
        x = self.dropout(self.embedding(x))
        out,(h_t,c_t) = self.LSTM(x)
#         print(x.shape,h_t.shape,c_t.shape)
#         s = torch.cat((h_t[0:1],h_t[1:2]),dim = 2)
#         print(s.shape)
        h_t = self.hidden(torch.cat((h_t[0:1],h_t[1:2]),dim = 2))
        c_t = self.cell(torch.cat((c_t[0:1],c_t[1:2]),dim = 2))
#         print(h_t.shape)
        return out,h_t,c_t
class LSTM_DECODER(nn.Module):
    def __init__(self,input_size,embedding_dim,hidden_dim,num_layers):
        super(LSTM_DECODER,self).__init__()
        self.input_size = input_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout = nn.Dropout(0.2)
        self.embedding = nn.Embedding(self.input_size,self.embedding_dim,padding_idx = 0)
        self.LSTM = nn.LSTM(self.embedding_dim,self.hidden_dim,num_layers = self.num_layers,dropout = 0.2,batch_first = True)
        self.fc = nn.Linear(self.hidden_dim,self.input_size)
    def forward(self,x,h_0,c_0):
        x = self.dropout(self.embedding(x))
        x = x.unsqueeze(1)
        out,(h_t,c_t) = self.LSTM(x,(h_0,c_0))
        out = self.fc(out)
        return out,h_t,c_t

In [ ]:
  
class LSTM_Att_DECODER(nn.Module):
    def __init__(self,input_size,embedding_dim,hidden_dim,num_layers):
        super(LSTM_Att_DECODER,self).__init__()
        self.input_size = input_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout = nn.Dropout(0.2)
        self.embedding = nn.Embedding(self.input_size,self.embedding_dim,padding_idx = 0)
        self.LSTM = nn.LSTM(2*self.hidden_dim+self.embedding_dim,self.hidden_dim,num_layers = self.num_layers,dropout = 0.2,batch_first = True)#recheck
        self.att = nn.Linear(3*self.hidden_dim,self.hidden_dim)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim = 1)#recheck
        self.energy = nn.Linear(self.hidden_dim,1,bias = False)
        self.fc = nn.Linear(self.hidden_dim,self.input_size)
    def forward(self,x,encoder_outs,h_t,c_t,att_mask = None):
        x = self.dropout(self.embedding(x))
        x = x.unsqueeze(1)
#         print(h_t.shape,encoder_outs.shape)
        h_0 = h_t
        h_t = h_t.repeat(encoder_outs.shape[1],1,1)
#         print(h_t.shape,"after")
#         print(encoder_outs.shape)
        h_t = h_t.transpose(0,1)
        energy = torch.cat([h_t,encoder_outs],dim = 2)
#         print(energy.shape)
        energy = self.att(energy)
        energy = self.relu(energy)
        energy = self.energy(energy)
#         energy = self.energy(self.relu(self.att(torch.cat([h_t,encoder_outs],dim = 2))))
        energy = energy.squeeze(2)
        
        attention = self.softmax(energy)
        attention = attention.unsqueeze(1)
        print(attention.shape,'hi')
        context = torch.bmm(attention,encoder_outs)
#         print(x.shape,context.shape)
        x = torch.cat([x,context],dim = 2)
#         print(x.shape)
        out,(ht,ct) = self.LSTM(x,(h_0,c_t))
        out = self.fc(out)
        return out,ht,ct

In [ ]:

class BERT_ENCODER(nn.Module):
    def __init__(self,frozen,tune_layers,hidden_units):
        super(BERT_ENCODER,self).__init__()
        self.frozen = frozen
        self.tune_layers = tune_layers
        self.hidden_units = hidden_units
        bert_model = BertModel.from_pretrained('bert-base-cased')
        if self.frozen == True:
            for name,params in bert_model.named_parameters():
                params.requires_grad = False
        else:
            if self.tune_layers!=-1:
                for params in bert_model.parameters():
                    params.requires_grad = False
                for params in bert_model.encoder.layer[-self.tune_layers:].parameters():
                    params.requires_grad = True
        self.encoder = bert_model
    def forward(self,x,att_mask):
        outs = self.encoder(input_ids = x,attention_mask = att_mask)
        encodings = outs.last_hidden_state
        hidden = None
        cell = None
        return encodings,hidden,cell
    
class BERT_Att_DECODER(nn.Module):
    def __init__(self,input_size,embedding_dim,hidden_dim,num_layers):
        super(BERT_Att_DECODER,self).__init__()
        self.input_size = input_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim  
        self.num_layers = num_layers
        self.dropout = nn.Dropout(0.2)
        self.embedding = nn.Embedding(self.input_size,self.embedding_dim,padding_idx = 0)#check
        self.LSTM = nn.LSTM(self.hidden_dim+self.embedding_dim,self.hidden_dim,num_layers = self.num_layers,dropout = 0.2,batch_first = True)
        self.att = nn.Linear(2*self.hidden_dim,1)
#         self.energy = nn.Linear(self.hidden_dim,1,bias = False)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim = 1)
        self.fc = nn.Linear(self.hidden_dim,self.input_size)
    def forward(self,x,encoder_outs,h_t,c_t):
        x = self.dropout(self.embedding(x))
        x = x.unsqueeze(1)
#         print(h_t.shape,encoder_outs.shape)
        h_0 = h_t
        h_t = h_t.repeat(encoder_outs.shape[1],1,1)
#         print(h_t.shape,"after")
#         print(encoder_outs.shape)
        h_t = h_t.transpose(0,1)
        energy = torch.cat([h_t,encoder_outs],dim = 2)
#         print(energy.shape)
        energy = self.att(energy)
        energy = self.relu(energy)
#         energy = self.energy(energy)
#         energy = self.energy(self.relu(self.att(torch.cat([h_t,encoder_outs],dim = 2))))
        energy = energy.squeeze(2)
        
        attention = self.softmax(energy)
        attention = attention.unsqueeze(1)
#         print(attention.shape)
        context = torch.bmm(attention,encoder_outs)
#         print(x.shape,context.shape)
        x = torch.cat([x,context],dim = 2)
#         print(x.shape)
        out,(ht,ct) = self.LSTM(x,(h_0,c_t))
        out = self.fc(out)
        
        return out,ht,ct

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self,encoder,decoder,is_att,tf_ratio = 0.6,target_vocab_size = len(vocab_Y)):
        super(Seq2Seq,self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.tf_ratio = tf_ratio
        self.target_vocab_size = target_vocab_size
        self.is_att = is_att
    def forward(self,problems,formulas,att_mask = None):
        batch_size = problems.shape[0]
        target_len = formulas.shape[1]
        target_vocab_size = self.target_vocab_size   
        outs,hidden,cell = self.encoder(problems,att_mask)
        if(cell == None):
#         outs = self.encoder(problems)
            hidden = torch.zeros(1, batch_size, self.decoder.hidden_dim).to(device)
            cell = torch.zeros(1, batch_size,self.decoder.hidden_dim).to(device)
#         print(target_len)
        outputs = torch.zeros(batch_size,target_len,target_vocab_size).to(device)
        x = formulas[:,0]
#         print(x)
        for i in range(1,target_len):
            if(self.is_att):
                output,hidden,cell = self.decoder(x,outs,hidden,cell)
            else:
                output,hidden,cell = self.decoder(x,hidden,cell)
            output = output.squeeze(1)
#             print(output.shape)
            outputs[:,i] = output
            x = output.argmax(dim = 1)
            if random.random() < self.tf_ratio:
                x = formulas[:,i]
        return outputs

In [ ]:
def get_encoder_decoder(model_type,input_size,output_size,hidden_dim,embedding_dim,num_layers,embedding_matrix):
    if(model_type == 'bilstm-lstm'):
        encoder = LSTM_ENCODER(input_size,embedding_dim,hidden_dim,num_layers,embedding_matrix)
        decoder = LSTM_DECODER(output_size,embedding_dim,hidden_dim,num_layers)
    elif(model_type == 'bilstm-att-lstm'):
        encoder = LSTM_ENCODER(input_size,embedding_dim,hidden_dim,num_layers,embedding_matrix)
        decoder = LSTM_Att_DECODER(output_size,embedding_dim,hidden_dim,num_layers)
    elif(model_type == 'frozen-bert-att-lstm'):
        encoder = BERT_ENCODER(True,-1,hidden_dim)
        decoder = BERT_Att_DECODER(output_size,embedding_dim,hidden_dim,num_layers)
    elif(model_type == 'bert-att-lstm'):
        encoder = BERT_ENCODER(True,1,hidden_dim)
        decoder = BERT_Att_DECODER(output_size,embedding_dim,hidden_dim,num_layers)
    return encoder,decoder

In [ ]:
os.makedirs('/kaggle/working/val')
os.makedirs('/kaggle/working/test')
os.makedirs('/kaggle/working/train')
os.makedirs('/kaggle/working/model')
os.makedirs('/kaggle/working/result')

In [ ]:
train_loader,val_loader,test_loader = data_loader(32)

In [ ]:
input_size = len(vocab_X)
output_size = len(vocab_Y)
hidden_dim = 256
embedding_dim = 300
num_layers = 1
glove = GloveEmbeddings(embedding_dim,word_to_idx_X)
embedding_matrix = glove.get_embedding()
max_len_X = max_len_X+2
max_len_Y = max_len_Y+2

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
    print('using device: cuda')
else:
    device = "cpu"
    print('using device: cpu')

In [ ]:


def train():
    models = ['bilstm-lstm','bilstm-att-lstm','frozen-bert-att-lstm','bert-att-lstm']
    encoder,decoder = get_encoder_decoder(models[1],input_size,output_size,hidden_dim,embedding_dim,num_layers,embedding_matrix)
    model = Seq2Seq(encoder,decoder,True)
#     model = nn.DataParallel(model)
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)
    
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer,50,verbose = False)
    
    loss_dict = defaultdict(list)
    accuracy_dict = defaultdict(list)
    
    best_epoch = 0
    least_loss = 1e9+7
    correct = 0
    correct_token = 0
    print('-----------------------------------Training------------------------------------')
    for epoch in range(50):
        model.train()
        print(f"\n----------------------------epoch: {epoch}----------------------------------")
        train_loss = []
        for idx,batch in enumerate(train_loader):
            optimizer.zero_grad()
#             print(batch)
            problems,formulas,ans = preprocess(batch)
            
            problems = problems.to(device)
            formulas = formulas.to(device)
#             att_mask = att_mask.to(device)
            outputs = model(problems,formulas)
            
            outputs = outputs.reshape(-1,outputs.shape[2])
            
            formulas = formulas.reshape(-1)
            
            loss = criterion(outputs,formulas)
            loss.backward()
            
            train_loss.append(loss.item())
            
            optimizer.step()
            # Assuming `obj` is the object consuming GPU memory
            obj = None

# Collect garbage
            gc.collect()

# Empty PyTorch cache
            torch.cuda.empty_cache()
#             _, predicted = torch.max(outputs, 2)
            # count correct if exact match
#             for pred, target in zip(predicted, formulas):
#                 if torch.equal(pred, target):
#                     correct += 1
#             total += formulas.size(0)
#             # count correct tokens
#             for pred, target in zip(predicted, targets):
#                 for p, t in zip(pred, target):
#                     if p == t:
#                         correct_token += 1
#                     total_token += 1
            
        scheduler.step()
#         accuracy = correct / total
        model.eval()
        
        val_loss = []
        print('-------------------------------------Validation---------------------------')
        for idx,batch in enumerate(val_loader):
            
            problems,formulas,ans = preprocess(batch)
            
            problems = problems.to(device)
            formulas = formulas.to(device)
#             att_mask = att_msk.to(device)
            output = model(problems,formulas)
            
            output = output.reshape(-1,output.shape[2])
            
            formulas = formulas.reshape(-1)
            
            loss = criterion(output,formulas)
            
            val_loss.append(loss.item())
            
        loss_val = np.mean(val_loss)
        loss_train = np.mean(train_loss)
        
        loss_dict['train'].append(loss_train)
        loss_dict['val'].append(loss_val)
            
        print(f"epoch:{epoch}, Train_loss:{loss_train}, Val_loss: {loss_val}")
        
        print('----saving checkpoint-----')
        with open(os.path.join('/kaggle/working/train','loss_dict_lstm-att3-lstm.json'),'w') as outfile:
            json.dump(loss_dict,outfile)
        torch.save(model,os.path.join('/kaggle/working/model','latest_model_lstm-att3-lstm.pth'))
        
        if(loss_val<least_loss):
            print(f"-------------updating least loss{loss_val} against{least_loss}-------------")
            least_loss = loss_val
            print('-----saving best model-----------')
            torch.save(model,os.path.join('/kaggle/working/model','best_model_lstm-att3-lstm.pth'))
    return 
        

In [ ]:
# train()

In [ ]:
# model = torch.load('/kaggle/input/latest-lstm-att-tf3/latest_model_lstm-att-lstm3.pth')

In [ ]:


# Assuming `obj` is the object consuming GPU memory
# obj = None

# # Collect garbage
# gc.collect()

# # Empty PyTorch cache
# torch.cuda.empty_cache()

In [ ]:
def beam_search(model,encoder_outs, enc_hidden, enc_cell, max_tgt_length=411, beam_size=10,att = True):
    beam = [([1], (enc_hidden, enc_cell), 0)]
    step = 0
    print('hi')
    while step < max_tgt_length - 1:
        new_beam = []
        for seq, (hidden, cell), score in beam:
            prev_token = [seq[-1]]
            prev_token = torch.LongTensor(prev_token).to(device)
#             print(hidden.shape,'hi')
            if(att):
                decoder_output, hidden, cell = model.decoder(prev_token,encoder_outs,hidden, cell)
            else:
                decoder_output, hidden, cell = model.decoder(prev_token, hidden, cell)
            decoder_output = decoder_output.squeeze(1)
            
            top_info = decoder_output.topk(beam_size, dim=1)
            top_values, top_indices = top_info
            
            for j in range(beam_size):
                new_word_index = top_indices[0][j]
                new_seq = seq + [new_word_index.item()]
                new_word_prob = torch.log(top_values[0][j])
                updated_score = score - new_word_prob
                new_candidate = (new_seq, (hidden, cell), updated_score)
                new_beam.append(new_candidate)
        new_beam.sort(key=lambda x: x[2])
        beam = new_beam[:beam_size]
        step += 1
#         print(beam)
    best_candidate = beam[0][0]
    decoded_words = torch.zeros(1, max_tgt_length)
#         print(best_candidate)
    for t in range(max_tgt_length):
        decoded_words[:, t] = torch.LongTensor([best_candidate[t]])
    return decoded_words


In [ ]:
def idx_to_sentence(output,idx_to_word):
    output = output.cpu().detach().numpy()
    preds = []
    for i in range(len(output)):
        p = []
        for idx in output[i]:
            w = idx_to_word[idx]
            if(w == "<end>"):
                break
            else:
                p.append(w)
        preds.append("".join(p[1:]))
#     preds = torch.tensor(preds)
    return preds

In [ ]:
def model_eval(model,idx_to_word,loader,m_id):
#     models = ['bilstm-lstm','bilstm-att-lstm','frozen-bert-att-lstm','bert-att-lstm']
#     m_id = 0
    beam = True
    results = []
    target_vocab_size = len(idx_to_word)
    outfile = []
    for idx,batch in enumerate(loader):
#         for i in range(len(batch)):
#             outfile.append({"Problem":batch["Problem"][i],"linear_formula":batch["linear_formula"][i],"answer":batch["answer"][i],"predicted":0})
        problems,formulas,ans = preprocess(batch)
        problems = problems.to(device)
        formulas = formulas.to(device)
#         att_mask = att_mask.to(device)
#         print(formulas.shape,'f')
        batch_size = problems.shape[0]
        max_target_len = formulas.shape[1]
        
        preds = torch.zeros(batch_size,max_target_len).to(device)
        encoder_outs,hidden,cell = model.encoder(problems)
#         print(hidden.shape,'hiii')
#         print(cell.shape,'byee')
#         print(encoder_outs.shape,'encodershape')
        if(cell == None):
#         outs = self.encoder(problems)
            hidden = torch.zeros(1, batch_size, model.decoder.hidden_dim).to(device)
            cell = torch.zeros(1, batch_size,model.decoder.hidden_dim).to(device)
        if(beam):
            for b in range(batch_size):
                if(m_id<2):
                    preds[b,:] = beam_search(model,encoder_outs[b,:,:].unsqueeze(0),hidden[:,b,:].unsqueeze(1),cell[:,b,:].unsqueeze(1),max_target_len,beam_size = 10,att = False)
                else:
                    preds[b,:] = beam_search(model,encoder_outs[b,:,:].unsqueeze(0),hidden[:,b,:].unsqueeze(1),cell[:,b,:].unsqueeze(1),max_target_len,beam_size = 1,att = True)
                    
        else:
            outputs = torch.zeros(batch_size,max_target_len,target_vocab_size).to(device)
            preds = torch.zeros(batch_size,max_target_len).to(device)
            x = formulas[:,0]
            preds[:,0] = formulas[:,0]
#             print(formulas[:,0])
            for t in range(1,max_target_len):
                output,hidden,cell = model.decoder(x,hidden,cell)
                output = output.squeeze(1)
                outputs[:,t,:] = output
                x = output.argmax(dim = 1)
                preds[:,t] = x
#             print(preds)
        predictions = idx_to_sentence(preds,idx_to_word)
#         for i in range(len(batch)):
#             outfile[i]["predicted"] = predictions[i]
        results.append(predictions)
        
    return results

In [ ]:
# results = model_eval(model,idx_to_word_Y,1)


In [ ]:
# for idx,batch in enumerate(train_loader):
#     print(batch['linear_formula'][5])
#     break

In [ ]:
# print(type(results))

In [ ]:
def create_outfile(model_file,loader,m_id):
    outfile = []
#     torch.backends.cuda.matmul.allow_tf32 = False  # Disables TF32 to improve compatibility
#     torch.backends.cudnn.benchmark = False  # Disable CuDNN benchmark mode
#     torch.backends.cudnn.deterministic = True
    model = torch.load(model_file)
        # Then move the model to the appropriate device
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
#     print(model)
    results = model_eval(model,idx_to_word_Y,loader,m_id)
    for idx,batch in enumerate(loader):
        for i in range(len(batch)):
            problem = batch["Problem"][i]
            linear_formula = batch["linear_formula"][i]
            answer = batch["answer"][i]
            predicted = results[idx][i]  # Convert to NumPy array and then to list if necessary
#             print(type(problem),type(linear_formula),type(answer),type(predicted))
            # Append serializable data to outfile
            answer = int(answer)
            outfile.append({"Problem": problem, "linear_formula": linear_formula, "answer": answer, "predicted": predicted})
#             outfile.append({"Problem":batch["Problem"][i],"linear_formula":batch["linear_formula"][i],"answer":batch["answer"][i],"predicted":results[idx][i]})
#     print(outfile)
    with open(os.path.join('/kaggle/working/result','results_lstm_test_beam.json'),'w') as file:
            json.dump(outfile,file)
    return outfile

In [ ]:
# print(results[0][5])
print('hi')

In [ ]:
outfile = create_outfile('/kaggle/input/latest-model-lstm/latest_model_lstm-lstm.pth',test_loader,1)
# print(outfile)

In [ ]:
# print(outfile)
# TORCH_USE_CUDA_DSA
# 

In [ ]:
# import json 
# import sys 
# import re 
# import math
# import pickle
# LPAR = '('
# RPAR = ')'
# ADD = 'add'
# SUB = 'subtract'
# DIV = 'divide'
# POW = 'power'
# MUL = 'multiply'
# NEG = 'negate'
# CAREA = 'circle_area'
# SAREA = 'square_area'
# RAREA = 'rectangle_area'
# VCYL = 'volume_cylinder'
# CONST = 'const_'
# FLOOR = 'floor'
# SQRT = 'sqrt'
# CIRCUM = 'circumface'
# INV = 'inverse'
# FACT = 'factorial'
# LOG = 'log'
# CHOOSE = 'choose'
# RHPERI = 'rhombus_perimeter'
# RPERI = 'rectangle_perimeter'
# QAREA = 'quadrilateral_area'
# SPEED = 'speed'
# MOD = 'reminder'
# VRPR = 'volume_rectangular_prism'
# PERMUT = 'permutation'
# ASPHERE = 'surface_sphere'
# TAREA = 'triangle_area'
# GCD = 'gcd'
# LCM = 'lcm'
# TPERI = 'triangle_perimeter'
# PGAIN = 'p_after_gain'
# TAREA3 =  'triangle_area_three_edges'
# CUBE_EDGE = 'cube_edge_by_volume'
# SRPR = 'surface_rectangular_prism'
# SQPERI = 'square_perimeter'
# MAX = 'max'
# SRCU = 'surface_cube'
# VCU = 'volume_cube'
# RHAREA =  'rhombus_area'
# OPRBLOSS = 'original_price_before_loss'
# SQUAE_EDGE =  'square_edge_by_area'
# VCONE = 'volume_cone'
# SS = 'stream_speed'
# SURF_CYL = 'surface_cylinder'
# VSPHERE = 'volume_sphere'
# OPRBGAIN =  'original_price_before_gain'
# MIN = 'min'
# SQR_EDGE_BY_PERI = 'square_edge_by_perimeter'
# NEG_PROB =  'negate_prob'

# #factorial function
# def factorial(n):
#     ans = 1
#     for i in range(1, int(n+1)):
#         ans *= i
#     return ans 

# #lcm function
# def lcm(a, b):
#     return abs(a * b) // math.gcd(a, b)

# #area of triangle
# def triangle_area(a, b, c):
#     # Calculate the semi-perimeter
#     s = (a + b + c) / 2
    
#     # Calculate the area using Heron's formula
#     area = math.sqrt(s * (s - a) * (s - b) * (s - c))
#     return area

# #the huge operation dict
# ops = {

#     ADD: lambda x,y: x+y,
#     SUB: lambda x,y: y-x,
#     DIV: lambda x,y: y/x,
#     POW: lambda x,y: y**x,
#     MUL: lambda x,y: x*y,
#     NEG: lambda x: -1*x,
#     CAREA: lambda x: (x**2)*math.pi,
#     VCYL: lambda x,y: 1/3*math.pi*(x**2)*y,
#     FLOOR: lambda x: int(x),
#     SQRT: lambda x: (x**0.5),
#     CIRCUM: lambda x: 2*math.pi*x,
#     SAREA: lambda x: x**2,
#     INV: lambda x: 1/x,
#     RAREA: lambda x,y: x*y,
#     FACT: factorial,
#     LOG: lambda x: math.log(x),
#     CHOOSE: lambda x,y: factorial(y)/(factorial(x)*factorial(y-x)),
#     RPERI: lambda x,y: 2*(x+y),
#     QAREA: lambda x,y,z: (y+x)/2*z, #new 15141
#     SPEED: lambda x,y: y/x,
#     MOD: lambda x,y: int(y)%int(x),
#     VRPR: lambda x,y,z: y*z*x,
#     RHPERI: lambda x: 4*x,
#     PERMUT: lambda x,y: factorial(y)/factorial(x),
#     ASPHERE: lambda x: 4*math.pi*(x**2),
#     TAREA: lambda x,y: (x*y)/2,
#     GCD: lambda x,y: math.gcd(int(y),int(x)),
#     LCM: lambda x,y: lcm(int(y), int(x)),
#     TPERI: lambda x,y,z: (x+y+z),
#     PGAIN: lambda x,y: y/x,
#     TAREA3: lambda x,y,z: triangle_area(x,y,z),
#     CUBE_EDGE: lambda x: x**(1/3),
#     SRPR: lambda x,y,z: 2*(x*y + y*z + z*x),
#     SQPERI: lambda x: 4*x,
#     MAX: lambda x,y: max(x,y),
#     SRCU: lambda x: 6*(x**2),
#     VCU: lambda x: 6*(x**2),
#     RHAREA: lambda x,y: (x+y)/2,
#     OPRBLOSS: lambda x,y: (100 - y)/100*x,
#     SQUAE_EDGE: lambda x: (x**0.5),
#     VCONE: lambda x,y: math.pi*(y**2)/3*y,
#     SS: lambda x,y: (x+y)/2,
#     SURF_CYL: lambda x,y: math.pi*2*x*y + 2*math.pi*(y**2),
#     VSPHERE: lambda x: 4/3*math.pi*(x**3),
#     OPRBGAIN: lambda x,y: (100+y)/100*x,
#     MIN: lambda x,y: min(x,y),
#     SQR_EDGE_BY_PERI: lambda x: x/4,
#     NEG_PROB: lambda x: (1-x)

# }

# #unitary operators
# uni_op = [NEG, SQRT, FLOOR, CAREA, CIRCUM, SAREA, INV, 
#             FACT, LOG, RHPERI, ASPHERE, CUBE_EDGE, SQPERI,
#             SRCU, VCU, SQUAE_EDGE, VSPHERE, 
#             SQR_EDGE_BY_PERI, NEG_PROB]

# #teranary operator
# tri_op = [QAREA, VRPR, TPERI, TAREA3, SRPR]

# #solve for one op
# def solve(op, args):
    
#     a1 = args[0]
#     #unary ?
#     if(op in uni_op):
#         ans = ops[op](a1)
    
#     #trinary operator
#     elif(op in tri_op):
#         a2 = args[1]
#         a3 = args[2]
#         ans = ops[op](a3, a2, a1)

#     #binary
#     else:
#         a2 = args[1]
#         ans = ops[op](a2, a1)
#     return ans

# #remove the commas
# pattern = r'\b\d{1,3}(,\d{3})+\b'

# # Function to remove commas from matched integers
# def remove_commas(match):
#     return match.group(0).replace(',', '')

# #get n0, n1, .. from problem statement
# def get_nums(sent):

#     #remove commas
#     global pattern
#     sent = re.sub(pattern, remove_commas, sent)

#     #match the digits
#     pt = r'[-+]?\d*\.\d+|\d+'
    
#     #sent = re.sub(sent, remove_commas, sent)
#     matches = re.findall(pt, sent)
#     numbers = [float(match) for match in matches]
#     return numbers

# #resolve the args
# def resolve_args(argsi, ans, nums):

#     #prev answers
#     if('#' in argsi):
#         argsi = argsi.split('#')[1].strip()
#         ret = ans[int(argsi)]

#     #some constant
#     elif('const_' in argsi):
#         argsi = argsi.strip()[6:]
#         if(argsi == 'pi'):
#             ret = math.pi 
#         else:
#             argsi = argsi.replace('_', '.')
#             ret = float(argsi)

#     #are they from input sentence
#     elif('n' in argsi):
#         w = int(argsi.strip()[1:])
#         ret = nums[w]
    
#     return ret 

# #evaluate linear formula
# def eval(x, sent):
    
#     #get number from sentences
#     nums = get_nums(sent)
    
#     #divide in the operations
#     ops = x.split('|')
#     ans = []

#     #iterate for every options
#     for oi in ops:

#         if(oi == ''):
#             continue

#         #split on (
#         t = oi.split('(')
        
#         #the operation
#         op = t[0]
        
#         #get the arg
#         args = t[1].split(')')[0]
#         args = args.split(',')
#         args = [argsi.strip() for argsi in args]

#         #resolve the args
#         #print(op, args)
#         args = [resolve_args(argsi, ans, nums) for argsi in args]

#         #get the ans
#         ansi = solve(op, args)
#         ans.append(ansi)

#     return ans[-1]

# #evaluate the linear expression and add to json
# def transform(data):

#     right = 0
#     wrong = 0
#     exact_match = 0
#     tr_data = []
    
#     for data_i in data:
        
#         #user predcition may not be executable
#         try: 
            
#             #copy data
#             tr_data_i = data_i

#             #the exact match acc
#             exact_match += (tr_data_i['predicted'] == tr_data_i['linear_formula'])

#             #evaluate
#             tr_data_i['predicted_answer'] = eval(data_i['predicted'], data_i['Problem'])

#             #print(data_i['annotated_formula'])
#             err = abs((tr_data_i['predicted_answer'] - tr_data_i['answer'] )/ tr_data_i['answer'])
            
#             #if beyond 2% then wrong
#             if(err > 0.02 or type(tr_data_i['predicted_answer']) == complex):
#                 wrong += 1
#                 tr_data_i['predicted_answer'] = None

#             #else match
#             else:
#                 right += 1
        
#             #return 
#             tr_data.append(tr_data_i)
#         except: 
            
#             tr_data_i['predicted_answer'] = None
#             wrong += 1
#             tr_data.append(tr_data_i)

#     print(f"Execution Accuracy: {right / (right + wrong)*100}!!" )
#     print(f"Exact Match Accuracy: {exact_match / (right + wrong)*100}!!" )
#     return tr_data

# def main(file):

#     f = open(file, 'rb')
#     data = json.load(f)

#     #convert the data
#     tr_data = transform(data)

#     #write back to file
#     f = open(file, 'w')
#     json.dump(tr_data, f, indent='\t', separators=(',', ': '))
# main("/kaggle/working/result/results_att_val_6.json")


# '''
# For the input json entry :

#     {
# 		"Problem": "a multiple choice test consists of 4 questions , and each question has 5 answer choices . in how many r ways can the test be completed if every question is unanswered ?",
# 		"answer": 625,
#         "linear_formula": "power(n1,n0)|"
# 	},

# The output json file entry should contain your prediction in predicted key
# and all other input fields:

#     {
# 		"Problem": "a multiple choice test consists of 4 questions , and each question has 5 answer choices . in how many r ways can the test be completed if every question is unanswered ?",
# 		"answer": 625,
#         "predicted": "power(n0,n0)|",
#         "linear_formula": "power(n1,n0)|"
# 	},

# We are counting on you to maintain the same files

# This script will write the predicted output back to output file


#     {
# 		"Problem": "a multiple choice test consists of 4 questions , and each question has 5 answer choices . in how many r ways can the test be completed if every question is unanswered ?",
# 		"answer": 625,
#         "predicted": "power(n0,n0)|",
#         "linear_formula": "power(n1,n0)|",
#         "predicted_answer": 256
# 	},
# '''